In [1]:
# Import libraries
from torch import nn,cuda,backends, optim, save, load, no_grad
import torch
from torchvision.datasets import ImageFolder, CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import time
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import pickle 
from pathlib import Path

In [2]:
# Image transformations
#mean=[0.6870, 0.5849, 0.5098]
#std=[0.2588, 0.3198, 0.3642]

#mean=[0.4914, 0.4822, 0.4465]
#std=[0.2023, 0.1994, 0.2010]

mean=[0.0178, 0.0574, 0.2181]
std=[1.0364, 1.0332, 1.0443]

transformations=transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomInvert(p=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=mean,std=std),
    ])

In [3]:
batch_size=50
# select classes you want to include in your subset
classes = torch.tensor([3, 8])
classes_names= ["cat","ship"]

data_train = CIFAR10(root='./data', train=True, download=True, transform=transformations)
indices = (torch.tensor(data_train.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
data_train = torch.utils.data.Subset(data_train, indices)
#classes_names=data_train.classes
dataloader_train = torch.utils.data.DataLoader(data_train,batch_size=batch_size, shuffle=True, num_workers=2)

data_test = CIFAR10(root='./data', train=False, download=True, transform=transformations)
indices = (torch.tensor(data_test.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
data_test = torch.utils.data.Subset(data_test, indices)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Get device for pytorch to run on
device = (
    "cuda"
    if cuda.is_available()
    else "mps"
    if backends.mps.is_available()
    else "cpu"
)
device="cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
x_train=[]
y_train=[]
for x_batch,y_batch in dataloader_train:
    for i in range(x_batch.shape[0]):
        x_train.append(x_batch[i].detach().numpy().flatten())
        y_train.append(y_batch[i].detach().numpy().flatten())
    
print((len(x_train),x_train[0].shape))
y_train=np.ravel(y_train)

(10000, (3072,))


In [6]:
x_test=[]
y_test=[]
for x_batch,y_batch in dataloader_test:
    for i in range(x_batch.shape[0]):
        x_test.append(x_batch[i].detach().numpy().flatten())
        y_test.append(y_batch[i].detach().numpy().flatten())
y_test=np.ravel(y_test)

In [11]:
epoches=8000
svms={}
Cs=[1,0.1,0.001]
dfs=["ovr"]
kernels=["linear", "poly", "rbf", "sigmoid"]
gammas=["auto","scale",0.001]
class_weights=[None]
degrees=[3,5]
tols=[0.001]
for tol in tols:
    for c in Cs:
        for df in dfs:
            for kernel in kernels:
                for class_weight in class_weights:
                    if (kernel=="poly"):
                        for degree in degrees:
                            for gamma in gammas:
                                svm_name="kernel_{0}/c_{1}/cw_{2}/dfs_{3}/tol_{4}/gamma_{5}/degree_{6}".format(kernel,c,class_weight,df,tol,gamma,degree)
                                svms[svm_name]=SVC(kernel=kernel, C=c,decision_function_shape=df,gamma=gamma,degree=degree,tol=tol,class_weight=class_weight, max_iter=epoches)
                                Path("/home/i/ioakeime/Models/svms/"+svm_name).mkdir(parents=True, exist_ok=True)
                    elif (kernel=="sigmoid" or kernel=="rbf"):
                        for gamma in gammas:
                            svm_name="kernel_{0}/c_{1}/cw_{2}/dfs_{3}/tol_{4}/gamma_{5}".format(kernel,c,class_weight,df,tol,gamma)
                            svms[svm_name]=SVC(kernel=kernel, C=c,decision_function_shape=df,gamma=gamma,tol=tol,class_weight=class_weight, max_iter=epoches)
                            Path("/home/i/ioakeime/Models/svms/"+svm_name).mkdir(parents=True, exist_ok=True)
                    else:
                        svm_name="kernel_{0}/c_{1}/cw_{2}/dfs_{3}/tol_{4}/".format(kernel,c,class_weight,df,tol)
                        svms[svm_name]=SVC(kernel=kernel, C=c,decision_function_shape=df,tol=tol,class_weight=class_weight, max_iter=epoches)
                        Path("/home/i/ioakeime/Models/svms/"+svm_name).mkdir(parents=True, exist_ok=True)
                        
                        
print(len(svms))
#svms["kernellinear_C1_dfsovr_maxiter"+str(epoches)]= SVC(kernel="linear", C=1.0,decision_function_shape="ovr", max_iter=epoches)
#svms["kernellinear_C0.1_dfsovr_maxiter"+str(epoches)]= SVC(kernel="linear", C=0.1,decision_function_shape="ovr", max_iter=epoches)

#svms["kernelsigmoid_C1_dfsovr_gammascale_maxiter"+str(epoches)]= SVC(kernel="sigmoid", C=1.0,decision_function_shape="ovr", max_iter=epoches,gamma="scale")
#svms["kernelsigmoid_C0.1_dfsovr_gammaauto_maxiter"+str(epoches)]= SVC(kernel="sigmoid", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma="auto")
#svms["kernelsigmoid_C0.1_dfsovr_gamma0.5_maxiter"+str(epoches)]= SVC(kernel="sigmoid", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma=0.5)
#svms["kernelsigmoid_C0.1_dfsovr_gammaauto_cwbalanced_maxiter"+str(epoches)]= SVC(kernel="sigmoid", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma="auto",class_weight="balanced")

#svms["kernelrbf_C1_dfsovr_gammascale_maxiter"+str(epoches)]= SVC(kernel="rbf", C=1.0,decision_function_shape="ovr", max_iter=epoches,gamma="scale")
#svms["kernelrbf_C0.1_dfsovr_gammaauto_maxiter"+str(epoches)]= SVC(kernel="rbf", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma="auto")
#svms["kernelrbf_C0.1_dfsovr_gamma0.5_maxiter"+str(epoches)]= SVC(kernel="rbf", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma=0.5)
#svms["kernelrbf_C0.1_dfsovr_gammaauto_cwbalanced_maxiter"+str(epoches)]= SVC(kernel="rbf", C=0.1,decision_function_shape="ovr", max_iter=epoches, gamma="auto",class_weight="balanced")



39


In [12]:
def calculate_class_success(cm):
    per_class_accuracies = {}
    string="Accuracies per class:\n"
    # Calculate the accuracy for each one of our classes
    for idx, cls in enumerate(classes_names):
        # True negatives are all the samples that are not our current GT class (not the current row) 
        # and were not predicted as the current class (not the current column)
        true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    
        # True positives are all the samples of our current GT class that were predicted as such
        true_positives = cm[idx, idx]
    
        # The accuracy for the current class is the ratio between correct predictions to all predictions
        per_class_accuracies[cls] = float((true_positives + true_negatives) / np.sum(cm))
        string+="Class: {0}, Accuracy: {1}\n".format(cls,per_class_accuracies[cls])
    return per_class_accuracies,string


In [13]:
max_accuracy=("",0)
log=open("/home/i/ioakeime/Models/svms/log.txt","w")
for key in svms.keys():
    model=svms[key]
    start_training=time.time()
    model.fit(x_train,y_train)
    stop_training=time.time()
    y_pred=model.predict(x_train)
    accuracy_train=np.sum(y_train==y_pred)/len(y_train)
    y_pred=model.predict(x_test)
    accuracy_test=np.sum(y_test==y_pred)/len(y_test)
    cm = confusion_matrix(y_test, y_pred)
    _,class_success_string=calculate_class_success(cm)
    if (accuracy_test>max_accuracy[1]):
        max_accuracy=(key,accuracy_test)
    pickle.dump(model, open("/home/i/ioakeime/Models/svms/"+key+"/model.sav", 'wb'))
    file=open("/home/i/ioakeime/Models/svms/"+key+"/log.txt", 'w')
    print("Model: {0}, Train Accuracy: {1}, Test Accuracy: {2}, Time trained:{3} seconds".format(key,accuracy_train,accuracy_test,stop_training-start_training),file=log)
    print("Confusion matrix:",cm,"\n",file=log)
    print(class_success_string,"\n",file=log)
    print('Iterations needed:', np.sum(model.n_iter_),"\n",file=log)
    print("Model: {0}, Train Accuracy: {1}, Test Accuracy: {2}, Time trained:{3} seconds".format(key,accuracy_train,accuracy_test,stop_training-start_training),file=file)
    print("Confusion matrix:",cm,file=file)
    print(class_success_string,file=file)
    print('Iterations needed:', np.sum(model.n_iter_),file=file)
    if (np.sum(model.n_iter_)<epoches):
        print("Converged {0} with {1} iterations\n".format(key,np.sum(model.n_iter_)),file=log)
        print("Converged {0} with {1} iterations\n".format(key,np.sum(model.n_iter_)),file=file)
        print("Converged {0} with {1} iterations\n".format(key,np.sum(model.n_iter_)))
    file.close()


log.close()
print("Best model:",max_accuracy)

/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/i/ioakeime/venvs/deeplearn

Converged kernel_rbf/c_1/cw_None/dfs_ovr/tol_0.001/gamma_scale with 6749 iterations



/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Converged kernel_sigmoid/c_1/cw_None/dfs_ovr/tol_0.001/gamma_auto with 5329 iterations

Converged kernel_sigmoid/c_1/cw_None/dfs_ovr/tol_0.001/gamma_scale with 6932 iterations

Converged kernel_sigmoid/c_1/cw_None/dfs_ovr/tol_0.001/gamma_0.001 with 3513 iterations



/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


...

Converged kernel_rbf/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_auto with 4094 iterations

Converged kernel_rbf/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_scale with 3955 iterations

Converged kernel_rbf/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_0.001 with 5175 iterations

Converged kernel_sigmoid/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_auto with 5631 iterations

Converged kernel_sigmoid/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_scale with 7463 iterations

Converged kernel_sigmoid/c_0.1/cw_None/dfs_ovr/tol_0.001/gamma_0.001 with 4785 iterations



/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Converged kernel_poly/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_auto/degree_3 with 4821 iterations

Converged kernel_poly/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_scale/degree_3 with 4972 iterations

Converged kernel_poly/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_0.001/degree_3 with 7179 iterations

Converged kernel_poly/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_auto/degree_5 with 5854 iterations

Converged kernel_poly/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_scale/degree_5 with 5000 iterations



/home/i/ioakeime/venvs/deeplearning/lib64/python3.9/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=8000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Converged kernel_rbf/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_auto with 5000 iterations

Converged kernel_rbf/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_scale with 5000 iterations

Converged kernel_rbf/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_0.001 with 5000 iterations

Converged kernel_sigmoid/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_auto with 5060 iterations

Converged kernel_sigmoid/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_scale with 5038 iterations

Converged kernel_sigmoid/c_0.001/cw_None/dfs_ovr/tol_0.001/gamma_0.001 with 5045 iterations

Best model: ('kernel_rbf/c_1/cw_None/dfs_ovr/tol_0.001/gamma_auto', np.float64(0.868))


In [9]:
# Calculate the mean and std of the dataset for normalization
mean = 0.
std = 0.
nb_samples = 0.
for data, _ in dataloader_train:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
#file=open("/Downloads/mean_std.txt","w")
print("Mean: "+str(mean)+"\n"+"Std: "+str(std))
#file.write("Mean: "+str(mean)+"\n"+"Std: "+str(std))
#file.close()

Mean: tensor([0.0205, 0.0601, 0.2227])
Std: tensor([1.0364, 1.0332, 1.0443])


In [7]:
with open('/home/i/ioakeime/Models/svms/kernel_rbf/c_1/cw_None/dfs_ovr/tol_0.001/gamma_auto/model.sav', 'rb') as f:
    model = pickle.load(f)
    start_time=time.time()
    y_pred=model.predict(x_test)
    accuracy_test=np.sum(y_test==y_pred)/len(y_test)
    stop_time=time.time()
    print("Test time:{0}".format(stop_time-start_time))

Test time:37.93816566467285
